In [ ]:
import pandas as pd

In [ ]:
import os
import pandas as pd

# Specify the folder containing your Excel files
data_folder = "Data"

# Define the column names
col_names = ['Case ID', 'Suspect Product Active Ingredients', #'Reason for Use', 
             'Serious', 'Sex', 'Patient Age', 'Patient Weight']

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame(columns=col_names)

# Iterate over all Excel files in the folder
for file in os.listdir(data_folder):
    if file.endswith(".xlsx"):  # Check if the file is an Excel file
        file_path = os.path.join(data_folder, file)
        # Read the Excel file and add it to the combined DataFrame
        data = pd.read_excel(file_path, usecols=col_names)  # Load only the specified columns
        combined_data = pd.concat([combined_data, data], ignore_index=True)

# Display the combined dataset
print(combined_data)

In [ ]:
col_names = ['Case ID', 'Suspect Product Active Ingredients', 'Reason for Use', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

# load dataset
#data = pd.read_excel("Data\\Line Listing(dxjZy).xlsx")

In [ ]:
data.head()

In [ ]:
selected_columns = ['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

#df = data[selected_columns]
df = combined_data[selected_columns]

df.head()
print(df)

In [ ]:
df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')
df_split = df.explode('Suspect Product Active Ingredients', ignore_index=True)

df_split.head()

In [ ]:
count = df_split['Serious'].value_counts()

print(count)

In [ ]:
print(df_split.dtypes)

In [ ]:
df_multi = pd.get_dummies(df_split, columns = ['Suspect Product Active Ingredients'], prefix='', prefix_sep='')

df_multi = df_multi.groupby('Case ID').max().reset_index()

columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

columns_to_convert = [col for col in df_multi.columns if col not in columns_to_exclude]

df_multi[columns_to_convert] = df_multi[columns_to_convert].astype(int)

df_final = df_multi
print(df_multi.columns)
print(df_final)

df_final.head()

In [ ]:
df_encoded = df_multi.copy()
print(df_encoded.columns)
#print(df_split['Patient Age'])
df_encoded['Patient Age'] = df_encoded['Patient Age'].astype(str)
df_encoded['Patient Age'] = df_encoded['Patient Age'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Age'] = pd.to_numeric(df_encoded['Patient Age'], errors='coerce')  # Converts to numeric, sets invalid values to NaN

df_encoded['Patient Weight'] = df_encoded['Patient Weight'].replace('Not Specified', "0 KG")
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].astype(str)
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'[^\d.]', '', regex=True)
df_encoded['Patient Weight'] = pd.to_numeric(df_encoded['Patient Weight'], errors='coerce')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

#df_encoded['Suspect Product Active Ingredients'] = label_encoder.fit_transform(df_encoded['Suspect Product Active Ingredients'])
df_encoded['Sex'] = label_encoder.fit_transform(df_encoded['Sex'])
df_encoded['Serious'] = label_encoder.fit_transform(df_encoded['Serious'])
print(df_encoded.isnull().sum())
df_encoded.dropna(inplace=True)
print(df_encoded.isnull().sum())
df_encoded.head()

In [ ]:
count = df_encoded['Serious'].value_counts()

print(count)

In [ ]:
cols_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Serious']

feature_cols = [col for col in df_encoded.columns if col not in cols_to_exclude]

print(feature_cols)
print(len(feature_cols))

X = df_encoded[feature_cols]
Y = df_encoded.Serious

In [ ]:
X.head()


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Address class imbalance using SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, Y)

# Standardize the data
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=26)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=26)

#!pip install imbalanced-learn

#from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install torch -q


In [ ]:

import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        #self.X = torch.from_numpy(X.to_numpy().astype(np.float32))
        #self.y = torch.from_numpy(y.to_numpy().astype(np.float32))
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.to_numpy().astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len
   
batch_size = 32

# Instantiate training and test data
train_data = Data(X_train, y_train)
#train_data = Data(X_resampled, y_resampled)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

valid_data = Data(X_val, y_val)
valid_dataloader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)

# Check it's working
for batch, (X, y) in enumerate(train_dataloader):
    print(f"Batch: {batch+1}")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    break


In [ ]:
import torch
from torch import nn
from torch import optim

input_dim = len(feature_cols) #393
hidden_dim = 1024 #512 #400
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        
        # Define multiple layers
        self.layer_1 = nn.Linear(input_dim, hidden_dim // 16)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        
        self.layer_2 = nn.Linear(hidden_dim // 16, hidden_dim // 32)  # Hidden layer (reduce size)
        nn.init.kaiming_uniform_(self.layer_2.weight, nonlinearity="relu")
        
        self.layer_3 = nn.Linear(hidden_dim // 32, hidden_dim // 64)  # Another hidden layer
        nn.init.kaiming_uniform_(self.layer_3.weight, nonlinearity="relu")

        self.dropout = nn.Dropout(p=0.2)
        
        self.output_layer = nn.Linear(hidden_dim // 64, output_dim)  # Final output layer
        
    def forward(self, x):
        # Pass through layers with ReLU activations
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.dropout(x)

        x = torch.nn.functional.relu(self.layer_2(x))
        x = self.dropout(x)

        x = torch.nn.functional.relu(self.layer_3(x))
        x = self.dropout(x)

        x = torch.nn.functional.sigmoid(self.output_layer(x))  # Sigmoid for binary output
        return x
       
model = NeuralNetwork(input_dim, hidden_dim, output_dim)
print(model)

In [ ]:
learning_rate = 0.01

loss_fn = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 15
train_loss_values = []
valid_loss_values = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
       
        # forward pass
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
        train_loss += loss.item()

        # backward pass and optimization
        loss.backward()
        optimizer.step()

    # average training loss for the epoch
    train_loss /= len(train_dataloader)
    train_loss_values.append(train_loss)

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val, y_val in valid_dataloader:
            pred = model(X_val)
            loss = loss_fn(pred, y_val.unsqueeze(-1))
            val_loss += loss.item()
    
    # Average validation loss for the epoch
    val_loss /= len(valid_dataloader)
    valid_loss_values.append(val_loss)

    # Print loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

print("Training Complete")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, num_epochs + 1), train_loss_values, label='Train Loss')
plt.plot(range(1, num_epochs + 1), valid_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import itertools  # Import this at the top of your script

# Initialize required variables
y_pred = []
y_test = []
correct = 0
total = 0

"""
We're not training so we don't need to calculate the gradients for our outputs
"""
with torch.no_grad():
    for X, y in test_dataloader:
        outputs = model(X)  # Get model outputs
        predicted = np.where(outputs.numpy() < 0.45, 0, 1)  # Convert to NumPy and apply threshold
        predicted = list(itertools.chain(*predicted))  # Flatten predictions
        y_pred.append(predicted)  # Append predictions
        y_test.append(y.numpy())  # Append true labels as NumPy
        total += y.size(0)  # Increment total count
        correct += (predicted == y.numpy()).sum().item()  # Count correct predictions

print(f'Accuracy of the network on the test instances: {100 * correct // total}%')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import seaborn as sns

y_pred = list(itertools.chain(*y_pred))
y_test = list(itertools.chain(*y_test))

print(classification_report(y_test, y_pred))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
cf_matrix

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Calculate ROC curve metrics
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Calculate AUC (Area Under the Curve)
roc_auc = roc_auc_score(y_test, y_pred)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="blue", label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")  # Diagonal line (random performance)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:

import numpy as np
import random
import torch

# Exclude specific columns
columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Sex', 'Patient Age', 'Patient Weight', 'Serious']

# Create drug-related columns
drug_cols = [col for col in df_encoded.columns if col not in columns_to_exclude]

# Generate random drug data
random_drugs = random.sample(drug_cols, 3)#["Insulin Pork\Insulin Purified Pork", "Insulin Beef"] #random.sample(drug_cols, 3)
print("Patient Drugs: ", random_drugs)
drug_array = np.zeros(len(drug_cols))
column_indices = [df_encoded.columns.get_loc(col) for col in random_drugs]
for idx in column_indices:
    drug_array[idx - 5] = 1  # Adjust index as needed

# Create a specific patient profile
specific_profile = np.array([[1, 83, 65]])  # Adjust profile values as needed
specific_profile = np.concatenate([specific_profile, drug_array.reshape(1, -1)], axis=1)

# Ensure the feature order matches the training data
#columns_to_include = [col for col in X.columns if col not in columns_to_exclude]  # 'X' should be your training feature matrix
#specific_profile = specific_profile[:, :len(columns_to_include)]  # Align shape with training features

# Standardize the profile using the same scaler used during training
specific_profile_scaled = scaler.transform(specific_profile)  # 'scaler' is the StandardScaler from training

# Convert the profile to a PyTorch tensor
specific_profile_tensor = torch.tensor(specific_profile_scaled, dtype=torch.float32)

# Predict the outcome using your PyTorch model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    prediction = model(specific_profile_tensor)
    predicted_outcome = (prediction >= 0.5).float().item()  # Threshold of 0.5
    predicted_probability = prediction.item()
print(predicted_outcome)
# Output the results
if predicted_outcome == 1.0:
    print("The reaction is classified as Serious.")
else:
    print("The reaction is classified as Non-Serious.")

print(f"Predicted Probability of Serious Reaction: {predicted_probability:.4f}")